# Main Program Testing
1. Inputs:
    * XMI Structure Model: .../Input/XMI/[....xml]
    * BOM: .../Input/BOM/Catalogs.xlsx
2. Outputs:
    * Excel: .../Output/InstanceTables/InstanceTable[date_time].xlsx
    * Log: .../Output/Logs/Log[date_time].txt

In [1]:
#Author: Habibi Husain Arifin
#Created Date: 10 January 2019
#Last Updated Date: 6 February 2020
#Version: 1.0
#<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

import datetime as DT
import xml.etree.ElementTree as ET
import copy
import pandas as PD

from deap import base, creator, tools, algorithms

#Custom API
from API.StructureModel import XMI25Element, Colors, XPathHelper, GeneticAlgorithmHelper
from API.RNLTree import RNLTree
from API.ExcelManager import ExcelManager
from API.Instance import Instance
from API.GeneticAlgorithms import GeneticAlgorithms
from API.Configuration import Configuration

#Change this one later
MOE = "MOE"

#File path
SOI_NAME = "Car"
#INPUT_XMI_FILE_PATH = "Input/XMI/car-variants-150%-190-sp2-v1.0-hbb.xml"
#INPUT_XMI_FILE_PATH = "Input/XMI/car-variants-100%-standard-190-sp2-v1.0-hbb.xml"
INPUT_XMI_FILE_PATH = "Input/XMI/car-variants-100%-sport-190-sp4-v1.0-hbb.xml"
INPUT_CATALOG_FILE_PATH = "Input/BOM/CarVariants-Catalogs.xlsx"
OUTPUT_EXCEL_FILE_PATH = "Output/InstanceTables/CarVariants-InstanceTable" + str(DT.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + ".xlsx" 
OUTPUT_LOG_FILE_PATH = "Output/Logs/CarVariants-Log" + str(DT.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + ".txt"  

"""
#Test Read StructureModel
"""
print("READ STRUCTURE MODEL")
tree = ET.parse(INPUT_XMI_FILE_PATH)
root = tree.getroot()
structModel = XMI25Element.createNewStructureModel(SOI_NAME, root)

#Print to file
#file = open(OUTPUT_FILE_PATH,"w")
for obj in structModel:
    print(Colors.getFontColorByLevel(obj.level), Colors.BOLD, XPathHelper.getRepeatedChar("", obj.level), obj.stereotypes, "LV.", obj.level, Colors.ENDC, ", gene =", obj.gaGene, ", nId =", obj.nodeId, ", pId =", obj.parentNodeId, ", xmiId =", obj.xmiId, ", name =", obj.name, ", value =", obj.value, ", formula =", obj.formula )
    #file.write("")
print("\nNo. of element:", len(structModel), "DateTime:", str(DT.datetime.now()))

#RNLTree.drawGraph(structModel)
RNLTree.drawSimplifiedGraph(structModel, root)

"""
#Test Read Instance from ExcelFile
"""
print("READ INSTANCES FROM CATALOG/BOM FILE")

instances, maxRowSize, maxColSize = ExcelManager.readExcelFile(INPUT_CATALOG_FILE_PATH)
instances = Instance.generateGeneSequentially(instances, maxRowSize)
print("MaxRowSize:", maxRowSize, "MaxColSize:", maxColSize)
for instance in instances:
    print(instance.insType, instance.gaGene, instance.parameters)
    
"""
#Test Genetic Algorithm
"""
#GAParameters
IND_SIZE = GeneticAlgorithmHelper.getIndividualSize(structModel) #We need to get it from the StructureModel
SIZE_OF_GENE = Instance.getSizeOfGene(maxRowSize) #We need to get it from the BOM
NO_OF_BEST_SOLUTIONS = 5 #Get it from the User UI
NO_OF_GENERATIONS = 100 #Get it from the User UI
POPULATION_SIZE = 10 #Get it from the User UI

print("RUN GENETIC ALGORITHMS")
toolbox = base.Toolbox()

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox.register("attr_alphabet", GeneticAlgorithms.generateGeneRandomly, k=SIZE_OF_GENE)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_alphabet, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", GeneticAlgorithms.evaluation, structureModel=structModel, instances=instances)

#Executes a one point crossover on the input sequence individuals.
#The two individuals are modified in place.
#The resulting individuals will respectively have the length of the other.
toolbox.register("mate", tools.cxOnePoint)

#Shuffle the attributes of the input individual and return the mutant.
#The individual is expected to be a sequence.
#The indpb argument is the probability of each attribute to be moved.
#Usually this mutation is applied on vector of indices.
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)

#Flip the value of the attributes of the input individual and return the mutant.
#The individual is expected to be a sequence and the values of the attributes shall stay valid after the not operator is called on them.
#The indpb argument is the probability of each attribute to be flipped.
#This mutation is usually applied on boolean individuals.
#toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)

#This function applies a gaussian mutation of mean mu and standard deviation sigma on the input individual.
#This mutation expects a sequence individual composed of real valued attributes.
#The indpb argument is the probability of each attribute to be mutated.
#toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)

toolbox.register("select", tools.selBest)

#Tool decoration is a very powerful feature that helps to control very precise things during an evolution
#without changing anything in the algorithm or operators.
chromosomeSeq = XMI25Element.getChromosomeSequence(structModel)
toolbox.decorate("mate", GeneticAlgorithms.checkExistence(chromosomeSeq, instances))
toolbox.decorate("mutate", GeneticAlgorithms.checkExistence(chromosomeSeq, instances))

pop = GeneticAlgorithms.customEvolution(POPULATION_SIZE, NO_OF_GENERATIONS, toolbox)
bests = tools.selBest(pop, k=NO_OF_BEST_SOLUTIONS)

#Print to file
file = open(OUTPUT_LOG_FILE_PATH,"w")
print("\n############################################################")
print("\n", chromosomeSeq, "\nSolution\tFitness")
file.write(":".join(chromosomeSeq) + "\nSolution\tFitness\n")
listChromosome = []
listFitness = []
listMOEs = []

for best in bests:
    print("\n", best, best.fitness)
    file.write("".join(best) + "\t" + str(best.fitness) + "\n")
    listChromosome.append("".join(best))
    listFitness.append(str(best.fitness))
    
    #Create configuration for Solution(s)
    sm = copy.deepcopy(structModel)
    bestConf = Configuration.createNewConfiguration(sm, best, instances) #createNewConfiguration(structureModel, chromosome, instances)
    moes = XMI25Element.getElementsByStereotype(bestConf.structureModel, MOE)
    dictMOE = {}
    for moe in moes:
        print(moe.name, ":", moe.value, moe.unitOnValueType)
        file.write(str(moe.name) + ":" + str(moe.value) + " " + str(moe.unitOnValueType) + "\n")
        dictMOE[str(moe.name) + "::" + str(moe.unitOnValueType)] = str(moe.value)
    listMOEs.append(dictMOE)
    dictMOE = {}
file.close()

#Write to excel
dicts = {}
dicts["Chromosome" + "[" + ",".join(chromosomeSeq) + "]"] = listChromosome
dicts["Fitness Value"] = listFitness
for listMOE in listMOEs:
    for key in listMOE.keys():
        if key not in dicts:dicts[key] = []
        dicts[key].append(listMOE[key])
df = PD.DataFrame(dicts)
ExcelManager.writeExcelFile(OUTPUT_EXCEL_FILE_PATH, df)

SyntaxError: invalid syntax (<ipython-input-1-5164477e1c10>, line 6)